In [ ]:
import lib.facebook_scraper as fs
import lib.utils as utils
import pandas as pd
import numpy as np
import os
import time

In [ ]:
post_ids_df = pd.read_csv('data/vebay69/clean/post_ids.csv')
post_ids_df.head()

In [ ]:
# Lấy danh sách post_id từ vị trí index của post_ids_df: pd.DataFrame với số lượng là number
def get_list_post_id(index: int, number: int) -> list:
    global post_ids_df
    if index < 0:
        index = 0
    if index > post_ids_df.shape[0]:
        return []
    if index + number > post_ids_df.shape[0]:
        return post_ids_df.iloc[index:]['post_id'].to_list()
    return post_ids_df.iloc[index:index+number]['post_id'].to_list()

In [ ]:
def extract_comment(comments, post_id):
    cnt_comment = 0
    cnt_reply = 0
    global comment_list
    for comment in comments:
        comment['post_id'] = post_id
        comment_list.append(comment)
        cnt_comment += 1
        time.sleep(np.random.uniform(0, 0.5))
        for reply in comment['replies']:
            reply['post_id'] = post_id
            comment_list.append(reply)
            cnt_reply += 1
            time.sleep(np.random.uniform(0, 0.5))

    print(f'\tExtracted {cnt_comment} comments and {cnt_reply} replies')
    return cnt_comment + cnt_reply

In [ ]:
index_post_id_file_path = os.path.join('data','vebay69','url','i_post_id_comment.txt')

### Rerun with change cookie

In [ ]:
fs.set_cookies('./cookies/cookie_1.txt')
# fs.enable_logging()

### Rerun

In [ ]:
NUMBER_POST = 30
post_list = []
comment_list = []
total_comment_real = 0

In [ ]:
index_post_id = utils.read_index_file(file_path=index_post_id_file_path)
print(index_post_id)
posts_ids = get_list_post_id(index_post_id, NUMBER_POST)
posts_ids

In [ ]:
try:
    for post in fs.get_posts(
        post_urls=posts_ids,
        options={
            "comments": True,
            "allow_extra_requests": True,
            "comments": "generator",
        },
        timeout=120,
    ):
        print(post)
        num_comment = extract_comment(post['comments_full'], post['original_request_url'])
        post_list.append(post)
        print(f'\tTotal {num_comment}/{post["comments"]} comments')
        total_comment_real += post['comments']
        utils.sleep(np.random.randint(30, 60))
except fs.exceptions.TemporarilyBanned:
    print("TEMPORARILY BANNED")
except fs.exceptions.AccountDisabled:
    print("ACCOUNT DISABLED")
except Exception as e:
    print(e)

In [ ]:
print(NUMBER_POST)
print(f"Số posts đã crawl: {len(post_list)}")
print(f"Số comments đã crawl: {len(comment_list)}/{total_comment_real}")

In [ ]:
if comment_list:
    utils.save_data(
        data_list=comment_list,
        folder_path=os.path.join('data','vebay69','raw'),
        type="comments"
    )
utils.write_index_file(
    file_path=index_post_id_file_path,
    index=index_post_id + len(post_list)
)